In [1]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
#get in the data
chinese = []
english_phoeneme = []
with open('dataset.txt') as f:
    for line in f:
        temp = line.split('\t')
        chinese.append(temp[1])
        english_phoeneme.append(temp[2])
english_phoeneme = [phoeneme.split('\n')[0].split() for phoeneme in english_phoeneme]
chinese = [list(word) for word in chinese]
for ele in chinese:
    if ' ' in ele:
        ele.remove(' ')
def prepare_vocab_input():
    vocab_inputs = []
    with open('./english_phoneme_vocabulary_output.txt') as file:
        for line in file:
            vocab_inputs.append(line.split('\n')[0])
    vocab_inputs.remove('_PAD')
    vocab_inputs.remove('_GO')
    vocab_inputs.remove('_EOS')
    vocab_inputs.remove('_UNK')
    vocab_inputs = ['PAD', 'EOS'] + (vocab_inputs)
    return vocab_inputs
def prepare_vocab_predict():
    vocab_predict = list(chinese_id_dict.keys())
    return vocab_predict
# prepare chinese_id_dict
chinese_id_list = []
with open('./chinese_vocabulary.txt') as file7:
    for line in file7:
        chinese_id_list.append(line.split('\n')[0])
chinese_id_list = ['PAD', 'EOS'] + chinese_id_list
chinese_id_dict = {}
for i in range(len(chinese_id_list)):
    chinese_id_dict[chinese_id_list[i]] = i
# finish preparation for chinese_id_dict
vocab_inputs = prepare_vocab_input()
vocab_predict = prepare_vocab_predict()
english_phoneme_dict = {}
for i in range(len(vocab_inputs)):
    english_phoneme_dict[vocab_inputs[i]] = i


In [174]:
for ele in chinese:
    if ' ' in ele:
        ele.remove(' ')

### Prepare everything

In [31]:
import numpy as np
import tensorflow as tf
import helpers
tf.reset_default_graph()
sess = tf.InteractiveSession()


### Intial some needed values

In [32]:
#128,256
batch_size = 256
embedding_size = 256
encoder_hidden_units = 100
decoder_hidden_units = 100
epoch = 100
encoder_max_length = 39
decoder_max_length =75

### Initial placeholders and variables

In [33]:
#placeholders pass data in
encoder_inputs = tf.placeholder(shape = (None, encoder_max_length), dtype = tf.int32)
decoder_targets = tf.placeholder(shape = (None, decoder_max_length+1), dtype = tf.int32)
decoder_inputs = tf.placeholder(shape = (None, decoder_max_length+1), dtype = tf.int32)

#variables
#size should be [max_time,batch_size] try batch_size here
encoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_inputs), embedding_size]
                                                   , -1.0, 1.0), dtype = tf.float32)
decoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_predict), embedding_size]
                                                   , -1.0, 1.0), dtype = tf.float32)
encoder_inputs_embedded = tf.nn.embedding_lookup(encoder_embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(decoder_embeddings, decoder_inputs)

### Encoder

In [34]:
#initial encoder cell to be LSTM
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell, encoder_inputs_embedded,
                                                dtype=tf.float32, time_major=False)

### Decoder

In [35]:
#initial decoder to be also LSTMcell
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(decoder_cell, decoder_inputs_embedded,
                            initial_state=encoder_final_state,dtype=tf.float32, time_major=False,scope='decoder')

### Connect layer and prediction

In [36]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, len(vocab_predict))
# [None, seq_len, vocab_size]

#在第二个维度上取argmax
decoder_prediction = tf.argmax(decoder_logits, 2)

### Optimizer

In [37]:
def sent_len(sentence):
    '''
    Args:
        sentence: [None, max_sentence_length]
    Returns:
        length: [None], the actual length of each sentence in the batch
    '''
    used = tf.sign(tf.abs(sentence))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(x=length, dtype=tf.int32)
    return length

In [387]:
len(vocab_predict) #116584 = 1534 *76

1534

In [388]:
decoder_max_length #5700 =76*75

75

In [389]:
decoder_targets

<tf.Tensor 'Placeholder_1:0' shape=(?, 76) dtype=int32>

In [390]:
sent_len(decoder_targets)

<tf.Tensor 'Sum:0' shape=(?,) dtype=int32>

In [38]:
cross_entropy_mask = tf.sequence_mask(
    lengths=sent_len(decoder_targets),
    maxlen=decoder_max_length + 1,
    dtype=tf.float32,
)
# [None, max_sentence_length]
cross_entropy_mask

<tf.Tensor 'SequenceMask/Cast_1:0' shape=(?, 76) dtype=float32>

In [39]:
loss = tf.contrib.seq2seq.sequence_loss(
    logits=decoder_logits, # [None, max_sentence_length, tag_set_size]
    targets=decoder_targets, # [None, max_sentence_length]
    weights=cross_entropy_mask,
    average_across_timesteps=True,
    average_across_batch=True,
    name="seq2seq_sequence_loss",
)
decoder_targets

<tf.Tensor 'Placeholder_1:0' shape=(?, 76) dtype=int32>

In [377]:
decoder_logits

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(?, 76, 1534) dtype=float32>

In [12]:
#use negative log likelihood to minimize
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels = tf.one_hot(decoder_targets, depth = len(vocab_predict), dtype = tf.float32),
    logits = decoder_logits)

#改weights
#seq2seq_entropy = tf.contrib.seq2seq.sequence_loss(logits = decoder_logits,targets = decoder_targets，)


loss = tf.reduce_mean(tf.reduce_sum(cross_entropy, 1))

In [40]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

sess.run(tf.global_variables_initializer())

### Train it

In [41]:
def batch(inputs, max_sequence_length = None):
    
    sequence_lengths = [len(seq) for seq in inputs]
    batch_size = len(inputs)
    
    if max_sequence_length is None:
        max_sequence_length = max(sequence_lengths)
    
    inputs_batch_major = np.zeros(shape = [batch_size, max_sequence_length], dtype = np.int32)
    
    for i, seq in enumerate(inputs):
        for j, element in enumerate(seq):
            inputs_batch_major[i, j] = element

    #inputs_time_major = inputs_batch_major.swapaxes(0, 1)

    return inputs_batch_major, sequence_lengths

In [42]:
def feeding(inputs, labels):
    inputs_int = []; predict_int = []
    for i in range(len(inputs)):
        single_input = []
        single_predict = []
        for x in range(encoder_max_length):
            try:
                single_input += [english_phoneme_dict[inputs[i][x]]]
            except:
                single_input += [0]
        for x in range(decoder_max_length):
            try:
                single_predict += [chinese_id_dict[labels[i][x]]]
            except:
                single_predict += [0]
        inputs_int.append(single_input); predict_int.append(single_predict)
            
    enc_input, _ = batch(inputs_int)
    dec_target, _ = batch([(sequence) + [1] for sequence in predict_int])
    dec_input, _ = batch([[1] + (sequence) for sequence in predict_int])
    #print(enc_input.shape)
    return {encoder_inputs: enc_input, decoder_inputs: dec_input, decoder_targets: dec_target}

In [320]:
feed = english_phoeneme[0:1],chinese[0:1]
feed

([['AH', 'B', 'IY', 'ER', 'D', 'EH', 'R']], [['阿', '伯', '德', '尔']])

In [397]:
feeding(english_phoeneme[0:1],chinese[0:1])

{<tf.Tensor 'Placeholder:0' shape=(?, 39) dtype=int32>: array([[ 2, 14,  7, 27,  9, 16,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0]], dtype=int32),
 <tf.Tensor 'Placeholder_2:0' shape=(?, 76) dtype=int32>: array([[  1, 872, 258, 489, 997,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32),
 <tf.Tensor 'Placeholder_1:0' shape=(?, 76) dtype=int32>: array([[872, 258, 489, 997,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0, 

In [ ]:
def comput_accuracy(inputs,feed_dict):
    predict = sess.run(decoder_prediction,feed_dict)

In [46]:
def label_to_chinese(label):
    chinese = ''
    for i in range(len(label)):
        for j in range(len(label[i])):
            if label[i][j] == 0 or label[i][j] == 1:
                continue
            chinese += vocab_predict[label[i][j]] + ' '
    return chinese

In [44]:
rand = 0
test = feeding(english_phoeneme[rand: rand + 1], chinese[rand: rand + 1])

In [48]:
predict = sess.run(decoder_prediction, test)
#label_to_chinese(predict)
predict

array([[872,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [43]:
import time
import random



for q in range(epoch):
    total_loss = 0
    lasttime = time.time()
    
    for w in range(0, len(english_phoeneme) - batch_size, batch_size):  
        
        #shuffle the batch
        combine = list(zip(english_phoeneme,chinese))
        random.shuffle(combine)
        batch_english_phoneme,batch_chinese = zip(*combine)
        
        _, losses = sess.run([optimizer, loss], 
                             feeding(batch_english_phoneme[w: w + batch_size], batch_chinese[w: w + batch_size]))
        
        total_loss += losses
        
    total_loss = total_loss / ((len(english_phoeneme) - batch_size) / (batch_size * 1.0))
    LOSS.append(total_loss)
    
    
    
    if (q + 1) % 10 == 0:
        print('epoch: ' + str(q + 1) + ', total loss: ' + str(total_loss) + ', s/epoch: ' + str(time.time() - lasttime))
    for i in range(10):
        rand = np.random.randint(len(english_phoeneme))
        test = feeding(english_phoeneme[rand: rand + 1], chinese[rand: rand + 1])
        predict = sess.run(decoder_prediction, test)
        print('input: ' + str(english_phoeneme[rand: rand + 1]))
        print('supposed label: ' + str(chinese[rand: rand + 1]))
        print('predict label:' + str(label_to_chinese(predict)) + '\n')
    print('#######################next 10 epoch#############################')

KeyboardInterrupt: 